In [26]:
"""
Created on Wed Mar 17 18:09:52 2021
@author: pimi
"""
#等待連線完成
#點選左側的資料夾圖示
#利用左側的上傳功能（上傳至工作階段儲存空間），上傳line匯出的對話txt檔
#編輯以下參數，分別為檔名，開始日期，結束日期，引號不可刪除
#點選左上方小箭頭執行程式
#右邊資料夾出現result.xlsx，下載即可

txt_name='[LINE] TLG的聊天.txt'
start_date='2021/04/01'
end_date='2021/04/30'


import pandas as pd
import os
import datetime
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

f=open(txt_name,'r',encoding='utf-8')
lines=f.readlines()
df=pd.DataFrame(lines)
df=df[0].str.split('\t',2,expand=True)
df.columns=['TIME','NAME','CONTENT']
searchfor=['（一','（二','（三','（四','（五','（六','（日','（週']
df['DATE']=df['TIME'][df['TIME'].str.contains('|'.join(searchfor))]
df['NAME'][~df['DATE'].isnull()]=0
df=df[df['NAME'].notnull()]
df['DATE']=df['DATE'].ffill()
df=df.iloc[60:]
df=df.dropna(subset=['CONTENT'])
df['DATE']=df['DATE'].str.split('（',expand=True)[0]
df['DATE']=pd.to_datetime(df['DATE'], format="%Y/%m/%d")
df['DATE']=df['DATE'].apply(lambda x: x.strftime('%Y/%m/%d'))
start_date=datetime.datetime.strptime(start_date, '%Y/%m/%d')
end_date=datetime.datetime.strptime(end_date, '%Y/%m/%d')
df=df[(df['DATE']>=start_date.strftime('%Y/%m/%d'))&(df['DATE']<=end_date.strftime('%Y/%m/%d'))]
#analysis
df_text=df[(df['CONTENT']!='[照片]\n')&(df['CONTENT']!='[貼圖]\n')]
df_photo=df[(df['CONTENT']=='[照片]\n')]
df_stick=df[(df['CONTENT']=='[貼圖]\n')]
#QTY count
def valueCount(df,colName):
    df_result=df[['NAME','CONTENT']].groupby('NAME').count().reset_index()
    df_result.columns=['NAME',colName]
    return df_result
df_count=valueCount(df,'ALL')
df_textCount=valueCount(df_text,'TEXT')
df_photoCount=valueCount(df_photo,'PHOTO')
df_stickCount=valueCount(df_stick,'STICK')

df_list=[df_count,df_textCount,df_photoCount,df_stickCount]
df_final = reduce(lambda left, right: pd.merge(left, right, on='NAME',how='left'), df_list)
df_final=df_final.fillna(0)
df_final=df_final[df_final['NAME']!=0].sort_values(by='ALL',ascending=False)
df_final.to_excel('result.xlsx',sheet_name='A',index=False)
#Day Count
df_dayCount=df[['DATE','CONTENT']].groupby('DATE').count().reset_index()
df_dayCount.columns=['DATE','QTY']
with pd.ExcelWriter('result.xlsx',engine="openpyxl",mode='a') as writer:
    df_dayCount.to_excel(writer, sheet_name='B',index=False)
